In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# NFL BIG DATA BOWL 2021
## Team Name :- Team06_5C_SoCSE_KLETech
## Team Members :-
#### Sanath A S - 01FE18BCS188 (368)
#### Raghavendra Kulkarni - 01FE18BCS162 (341)
#### Rohan Sirigeri - 01FE18BCS176 (356)
#### Ruturaj Chitragar - 01FE18BCS180 (380)

In [ ]:
## Import the Libraries

## Start Code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline
import matplotlib.patches as patches
from glob import glob
import seaborn as sns
import plotly.express as px
## End Code

# The Data Set
## The dataset contains the following comma seperated values (.csv) files :-
### 1. games.csv :- Teams playing in each game and game details. The key variable is gameId.
### 2. plays.csv :- Play information from each game. The key variables are gameId and playId.
### 3. players.csv :- Information of the players participating in the NFL Season. The key variable is nflId.
### 4. week[i].csv :- Player tracking data from all games in week i. The key variables are gameId, playId, 

### and nflId. There are 17 weeks to a typical NFL Regular Season, and thus 17 data frames with player tracking 

### data are provided.

### The dataset contains a total of 1 + 1 + 1 + (1 x 17) = 20 csv files.
## Bonus Data :- An extra csv file is also used in the below analysis.
### targettingReceivers.csv :- nflId of the player to whom the pass in a particular play was made. The key 

### variables are gameId and playId.

## I. Pre-Processing and Exploring the Input csv files

### 1. Read and Pre-Process the games.csv File

In [ ]:
## Read the games.csv file and store it in a pandas dataframe

##Start Code
gamesData=pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv')
##End Code

gamesData

In [ ]:
## Check the size of the dataframe
gamesData.shape

In [ ]:
## Check the size of the dataframe
gamesData.shape

In [ ]:
## Check the attributes and their datatypes
gamesData.info()

In [ ]:
## Count the null values in each attribute
gamesData.isnull().sum()

In [ ]:
## Count unique values in each attribute
gamesData.nunique()

In [ ]:
## Combine the gameDate and gameTimeEastern as gameTimeEastern and convert it into pandas dateTime, dropping the gameDate.

## Start Code
gamesData['gameTimeEastern']=gamesData['gameDate'] + ' ' + gamesData['gameTimeEastern']
gamesData.gameTimeEastern=pd.to_datetime(gamesData.gameTimeEastern)
gamesData=gamesData.set_index('gameTimeEastern')
gamesData=gamesData.tz_localize('EST').tz_convert('EST')
gamesData=gamesData.reset_index()
gamesData=gamesData.drop('gameDate',axis=1)
## End Code

gamesData

In [ ]:
#### How many games have each time played in their home ground ?

## Start Code
teams=list(set(list(gamesData.homeTeamAbbr.unique())+list(gamesData.visitorTeamAbbr.unique())))
gamesAtHome=pd.DataFrame(gamesData.groupby('homeTeamAbbr')['gameId'].count())
gamesAtHome.reset_index(inplace=True)
plt.figure(figsize=(20,5))
plt.bar(gamesAtHome.homeTeamAbbr,gamesAtHome.gameId)
plt.xlabel('Teams')
plt.ylabel('Number of Games')
plt.savefig('Output1.jpg')
plt.show()
##End Code

#### Almost all teams have played 8 games in their home ground

#### How many games have each team played in opponent's ground ?

In [ ]:
## Start Code
gamesAtHome=pd.DataFrame(gamesData.groupby('visitorTeamAbbr')['gameId'].count())
gamesAtHome.reset_index(inplace=True)
plt.figure(figsize=(20,5))
plt.bar(gamesAtHome.visitorTeamAbbr,gamesAtHome.gameId)
plt.xlabel('Teams')
plt.ylabel('Number of Games')
plt.show()
plt.savefig('Output2.jpg')
## End Code

#### Almost all teams have played 8 games in their opponent ground

## 2. Read and Pre-Process the plays.csv File

In [ ]:
## Read the plays.csv file and store it in a pandas dataframe

## Start Code
playsData=pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
## End Code

playsData

In [ ]:
## Check the size of the dataframe
playsData.shape

In [ ]:
## Check the attributes and their data types
playsData.info()

In [ ]:
## Count null values in each attribute
playsData.isnull().sum()

In [ ]:
## Fill null values in yardlineSide with 'CENTRE' as the yardlineNumber is 50 for them

## Start Code
playsData.loc[playsData.yardlineSide.isnull(),'yardlineSide']='CENTRE'
## End Code

playsData.isnull().sum()

In [ ]:
# Fill the null values of offenseFormation from playDescription attribute

## Start Code
playsData.offenseFormation=playsData.offenseFormation.replace('EMPTY',np.nan)
playsData.offenseFormation.fillna(playsData.playDescription,inplace=True)
playsData.offenseFormation=playsData.offenseFormation.str.split(n=1).str[-1]
playsData.offenseFormation = playsData.offenseFormation.str.split(')', n = 1, expand = True)
playsData.offenseFormation = playsData.offenseFormation.apply(lambda x:x.strip('('))
playsData.offenseFormation=playsData.offenseFormation.replace('Shotgun','SHOTGUN')
playsData.offenseFormation=playsData.offenseFormation.replace('No Huddle','NO HUDDLE')
playsData.offenseFormation=playsData.offenseFormation.replace('No Huddle, Shotgun','NO HUDDLE, SHOTGUN')
def clean(x):
    if x == 'I_FORM' or x == 'SINGLEBACK' or x == 'SHOTGUN' or x == 'PISTOL' or x == 'WILDCAT' or x == 'NO HUDDLE' or x == 'JUMBO' or x == 'NO HUDDLE, SHOTGUN':
        return x
    else: 
        return np.nan
playsData.offenseFormation = playsData.offenseFormation.apply(clean)
## End Code

In [ ]:
## Fill the null values of defendersInTheBox,numberOfPassRushers,preSnapVisitorScore,preSnapHomeScore attributes with mean of their respective bins

## Start Code
playsData.defendersInTheBox = playsData.defendersInTheBox.fillna(playsData.groupby('playId')['defendersInTheBox'].transform('mean'))
playsData.numberOfPassRushers = playsData.numberOfPassRushers.fillna(playsData.groupby('playId')['numberOfPassRushers'].transform('mean'))
playsData.preSnapVisitorScore = playsData.preSnapVisitorScore.fillna(playsData.groupby('playId')['preSnapVisitorScore'].transform('mean'))
playsData.preSnapHomeScore = playsData.preSnapHomeScore.fillna(playsData.groupby('playId')['preSnapHomeScore'].transform('mean'))
## End Code

In [ ]:
## Extract the missing data in gameClock from playDescription

## Start Code
playsData['gameClock']=playsData.playDescription.str.split(')',n=1,expand=True)
playsData.gameClock=playsData.gameClock.apply(lambda x:x.replace('(:','00:'))
playsData.gameClock=playsData.gameClock.apply(lambda x:x.strip('('))
playsData.gameClock=pd.to_datetime(playsData.gameClock)
## End Code

In [ ]:
## Fill the null values of absoluteYardlineNumber attribute with mean of the respective bins

## Start Code
playsData.absoluteYardlineNumber = playsData.absoluteYardlineNumber.fillna(playsData.groupby('playId')['absoluteYardlineNumber'].transform('mean'))
## End Code

In [ ]:
## Drop the unrequired attributes

## Start Code
playsData.loc[playsData['penaltyCodes'].isnull(),'penaltyCodes']='None'
playsData.loc[playsData['penaltyJerseyNumbers'].isnull(),'penaltyJerseyNumbers']='None'
## End Code

In [ ]:
## Compute the penalty on each team in every play

## Start Code
playsData['penaltyOnDefense']=playsData['playResult']-playsData['offensePlayResult']
playsData['penaltyOnOffense']=playsData['offensePlayResult']-playsData['playResult']
playsData.loc[playsData.penaltyOnDefense<0,'penaltyOnDefense']=0
playsData.loc[playsData.penaltyOnOffense<0,'penaltyOnOffense']=0
## End Code

In [ ]:
## Check the null values in each attribute
playsData.isnull().sum()

## 3. Read and Pre-Process the players.csv File

In [ ]:
## Read the games.csv file and store it in a pandas dataframe

##Start Code
playersData=pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv')
## End Code

playersData

In [ ]:
## Count the null values in each attribute

playersData.isnull().sum()

In [ ]:
## Convert the data type of the height attribute from string to integer

## Start Code
def cleanHeight(x):
    if len(x)==2:
        return int(x)
    else:
        return np.nan
playersData.height=playersData.height.apply(cleanHeight)
## End Code

In [ ]:
## Check the null values after this conversion
playersData.height.isnull().sum()

In [ ]:
## Fill the null values of height with mean of their respective bins of weight

## Start Code
playersData.height=playersData.height.fillna(playersData.groupby('position')['height'].transform('mean'))
## End Code

playersData.height.isnull().sum()

#### How is the height of the players distributed ?

In [ ]:
## Plot a Histogram of height distribution of the players

## Start Code
bins = np.arange(playersData.height.min(),playersData.height.max() + 1, 1)
sns.distplot(playersData.height,bins = bins)
plt.savefig('Output3.jpg')
## End Code

#### The height is distributed almost normally. Hence the filling of null values has not tampered the distribution.

In [ ]:
## Check for other null valued attributes
playersData.isnull().sum()

In [ ]:
## Convert birthDate attribute to pandas date and time format

## Start Code
playersData.birthDate=pd.to_datetime(playersData.birthDate)
## End Code

playersData.info()

In [ ]:
## Compute the age of the players for analysing it against their performance

## Start Code
def compute(x):
    return 2018-x.year

playersData['age']=playersData.birthDate.apply(compute)
## End Code

playersData

#### Who are the NFL's top 10 tallest players ?

In [ ]:
## Plot top 10 players Height Distribution

## Start Code
heightCount=pd.DataFrame(playersData.groupby('height')['displayName'].value_counts().sort_values(ascending=False))
heightCount=heightCount.drop('displayName',axis=1).reset_index()
fig,ax1=plt.subplots(1,figsize=(10,5))
p=sns.barplot(x=heightCount.height,y=heightCount.displayName[:10],ax=ax1,linewidth=1,alpha=0.7,palette='Blues_r')
for i,j in enumerate(heightCount.height[:10]):
        ax1.text(.7,i+0.08,'{:0.1f}'.format(j),weight="bold",size=15)
p.set_title("Top Players Height Distribution",weight='bold',size=15)
p.set_xlabel('Player Height (Inches)',size=15)
p.set_ylabel('Player\'s  Distribution',size=15)
plt.show()
plt.savefig('Output4.jpg')
## End Code

#### Margus Hunt is the tallest among all the NFL Players

#### Who are the NFL's top 10 oldest players ?

In [ ]:
## Plot top 10 players Age Distribution

## Start Code
ageCount=pd.DataFrame(playersData.groupby('age')['displayName'].value_counts())
ageCount=ageCount.drop('displayName',axis=1).reset_index()
ageCount=ageCount.sort_values('age',ascending=False)
fig,ax1=plt.subplots(1,figsize=(10,5))
p=sns.barplot(x=ageCount.age,y=ageCount.displayName[:10],ax=ax1,linewidth=1,alpha=0.7,palette='Blues_r')
for i,j in enumerate(ageCount.age[:10]):
        ax1.text(.7,i+0.08,'{:0.1f}'.format(j),weight="bold",size=15)
p.set_title("Top Players Age Distribution",weight='bold',size=15)
p.set_xlabel('Player Age',size=15)
p.set_ylabel('Player\'s  Distribution',size=15)
plt.show()
plt.savefig('Output5.jpg')
## End Code

#### Tom Brady is the oldest NFL player aged 41

#### How many players are there in each player position ?

In [ ]:
## Start Code
positionCount=pd.DataFrame(playersData.groupby('position')['nflId'].count())
positionCount.reset_index(inplace=True)
positionCount=positionCount.sort_values('nflId',ascending=False)
positionCount.reset_index(inplace=True)
plt.figure(figsize=(20,5))
plt.bar(positionCount.position,positionCount.nflId)
plt.xlabel('Position')
plt.ylabel('Number of Players')
plt.show()
plt.savefig('Output6.jpg')
## End code

#### Wide Receivers (WB) are most in common whereas, HalfBack (HB) is the least occuring position among the players.

#### Wide Receivers are the strongest part of any offense team and they must be the min target of any defensive success.

#### Is there any correlation between the average height and weight of the players in each player position ?

In [ ]:
## Start Code
positionHW=pd.DataFrame(playersData.groupby('position')[['height','weight']].mean())
positionHW.reset_index(inplace=True)
plt.figure(figsize=(10,5))
plt.scatter(positionHW.weight,positionHW.height)
plt.xlabel('Weight (in pounds)')
plt.ylabel('Height (in inches)')
plt.show()
plt.savefig('Output7.jpg')
## End Code

#### There exists no such strong correlation between the average height and weight of the players in a position.

## 4. Read and Pre-Process the week[i].csv Files, i ranging from [0,17)

In [ ]:
## List the names of all the 17 weeks.csv files

## Start Code
weeks=[
    '../input/nfl-big-data-bowl-2021/week1.csv',
    '../input/nfl-big-data-bowl-2021/week2.csv',
    '../input/nfl-big-data-bowl-2021/week3.csv',
    '../input/nfl-big-data-bowl-2021/week4.csv',
    '../input/nfl-big-data-bowl-2021/week5.csv',
    '../input/nfl-big-data-bowl-2021/week6.csv',
    '../input/nfl-big-data-bowl-2021/week7.csv',
    '../input/nfl-big-data-bowl-2021/week8.csv',
    '../input/nfl-big-data-bowl-2021/week9.csv',
    '../input/nfl-big-data-bowl-2021/week10.csv',
    '../input/nfl-big-data-bowl-2021/week11.csv',
    '../input/nfl-big-data-bowl-2021/week12.csv',
    '../input/nfl-big-data-bowl-2021/week13.csv',
    '../input/nfl-big-data-bowl-2021/week14.csv',
    '../input/nfl-big-data-bowl-2021/week15.csv',
    '../input/nfl-big-data-bowl-2021/week16.csv',
    '../input/nfl-big-data-bowl-2021/week17.csv'
]
## End Code

In [ ]:
## Read all the 17 week files and store them into a list of dataframes

## Start Code
weeksData=[]
for week in weeks:
    weeksData.append(pd.read_csv(week))
## End Code

print(str(len(weeksData))+' weeks Data read')

In [ ]:
## Convert the time attribute to pandas dataframe, eliminate tuples with missing player information and set null values of
## route attribute to 'UNKNOWN' in each week file

## Start Code
def preProcess(weekData):
    weekData.time=pd.to_datetime(weekData.time)
    weekData=weekData.set_index('time')
    weekData=weekData.tz_convert('EST')
    weekData=weekData.reset_index()
    weekData.drop(weekData[weekData.team=='football'].index,inplace=True)
    weekData.loc[weekData['route'].isnull(),'route']='UNKNOWN'
    return weekData

for i in range(17):
    weeksData[i]=preProcess(weeksData[i])
## End Code

## 5. Read and Pre-process the targettedReceiver.csv file.

In [ ]:
## Read the targettedReceiver.csv file and store it in a pandas dataframe

##Start Code
targettedReceiversData=pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/targetedReceiver.csv')
## End Code

targettedReceiversData

In [ ]:
## Drop the columns where there is no specific targetted receiver. Compute the week in each game was played 
### and attach it as a new column to easily access the tracking data file of that respective week.

## Start Code
def func(Id):
    x=gamesData.loc[gamesData.gameId==Id,'week']
    return int(x)
targettedReceiversData.drop(targettedReceiversData[targettedReceiversData.targetNflId.isnull()].index,inplace=True)
targettedReceiversData['weekNum']=targettedReceiversData.gameId.apply(func)
targettedReceiversData=targettedReceiversData.rename(columns={'targetNflId':'nflId'})
## End Code

targettedReceiversData

## II. The Potential Player Profile Approach

### 1. Create Player Profiles

In [ ]:
## Keep a list of all the defensive positions to retrieve defensive players only

## Start Code
defensivePositions=['CB','SS','MLB','OLB','FS','DE','LB','ILB','DB','S','NT','DT']
## End Code

In [ ]:
## Copy Players Data into another Dataframe to create player profiles of defensive players only

## Start Code
playerProfile=playersData.copy()
playerProfile=playerProfile[playerProfile['position'].isin(defensivePositions)]
## End Code

In [ ]:
## Create a class to store the attributes

## Start Code
players=[]
class Player:
    def __init__(self,id):
        self.nflId=id
        self.maxSpeed=0
        self.maxAccl=0
    def display(self):
        print(str(self.nflId)+' '+str(self.maxSpeed))
## End Code

In [ ]:
## Compute the maximumSpeed and maximumAcceleration for each player across all the 17 weeks of the NFL season 2018

## Start Code
maxSpeeds=[]
maxAccls=[]
for weekData in weeksData:
    maxSpeeds.append(weekData.groupby('nflId')['s'].max())
    maxAccls.append(weekData.groupby('nflId')['a'].max())

def createObjects(id,pos):
    if pos in defensivePositions:
        players.append(Player(id))

playersData.apply(lambda row : createObjects(row['nflId'],row['position']),axis=1)
for ele in players:
    for i in range(17):
        try:
            ele.maxSpeed=max(ele.maxSpeed,maxSpeeds[i][ele.nflId])
            ele.maxAccl=max(ele.maxAccl,maxAccls[i][ele.nflId])
        except:
            ele.maxSpeed=max(ele.maxSpeed,0)
            ele.maxAccl=max(ele.maxAccl,0)
## End Code

In [ ]:
## List the events of weekData to detect defensive acts by the players

## Start Code
defensiveEvents=[]
for weekData in weeksData:
    defensiveEvents+=list(weekData.event.unique())
defensiveEvents=list(set(defensiveEvents))
## End Code

defensiveEvents

#### We consider the events 'tackle' , 'pass_outcome_interception' , 'qb_sack' and 'out_of_bounds', which respectively represent a
#### tackle, a pass interception, a quarterback sack and a out of bounds push.

In [ ]:
## Compute the total number of tackles by each player across all the 17 weeks of the NFL season 2018

## Start Code
tackles=[]
for weekData in weeksData:
    tackle=weekData.copy()
    tackle.drop(tackle[tackle.event!='tackle'].index,inplace=True)
    tackle=tackle[tackle['position'].isin(defensivePositions)]
    tackles.append(tackle)
tacklesData=pd.concat(tackles)
playerWiseTotalTackles=tacklesData.groupby('nflId')['event'].count()
for ele in players:
    try:
        ele.tackles=playerWiseTotalTackles[ele.nflId]
    except KeyError:
        ele.tackles=0
## End Code

In [ ]:
## Compute the total number of quarterback sacks by each player across all the 17 weeks of the NFL season 2018

## Start Code
qbSacks=[]
for weekData in weeksData:
    qbSack=weekData.copy()
    qbSack.drop(qbSack[qbSack.event!='qb_sack'].index,inplace=True)
    qbSack=qbSack[qbSack['position'].isin(defensivePositions)]
    qbSacks.append(qbSack)
qbSacksData=pd.concat(qbSacks)
playerWiseTotalqbSacks=qbSacksData.groupby('nflId')['event'].count()
for ele in players:
    try:
        ele.qbSacks=playerWiseTotalqbSacks[ele.nflId]
    except KeyError:
        ele.qbSacks=0
## End Code

In [ ]:
## Compute the total number of Pass Interceptions by each player across all the 17 weeks of the NFL season 2018

## Start Code
interceptions=[]
for weekData in weeksData:
    interception=weekData.copy()
    interception.drop(interception[interception.event!='pass_outcome_interception'].index,inplace=True)
    interception=interception[interception['position'].isin(defensivePositions)]
    interceptions.append(interception)
interceptionsData=pd.concat(interceptions)
playerWiseTotalInterceptions=interceptionsData.groupby('nflId')['event'].count()
for ele in players:
    try:
        ele.interceptions=playerWiseTotalInterceptions[ele.nflId]
    except KeyError:
        ele.interceptions=0
## End Code

In [ ]:
## Compute the total number of 'Out of Bounds Push' by each player across all the 17 weeks of the NFL season 2018

## Start Code
outOfBounds=[]
for weekData in weeksData:
    outOfBound=weekData.copy()
    outOfBound.drop(outOfBound[outOfBound.event!='out_of_bounds'].index,inplace=True)
    outOfBound=outOfBound[outOfBound['position'].isin(defensivePositions)]
    outOfBounds.append(outOfBound)
outOfBoundsData=pd.concat(outOfBounds)
playerWiseTotalOutOfBounds=outOfBoundsData.groupby('nflId')['event'].count()
for ele in players:
    try:
        ele.outOfBounds=playerWiseTotalOutOfBounds[ele.nflId]
    except KeyError:
        ele.outOfBounds=0
## End Code

In [ ]:
## Append these values to the playerProfile DataFrame to prepare the Player Profiles

## Start Code
def setMaxSpeed(x):
    for ele in players:
        if ele.nflId==x:
            return ele.maxSpeed
    
def setMaxAcclr(x):
    for ele in players:
        if ele.nflId==x:
            return ele.maxAccl
    
def setTackle(x):
    for ele in players:
        if ele.nflId==x:
            return ele.tackles
        
def setqbSack(x):
    for ele in players:
        if ele.nflId==x:
            return ele.qbSacks
    
def setinterceptions(x):
    for ele in players:
        if ele.nflId==x:
            return ele.interceptions

def setOutOfBounds(x):
    for ele in players:
        if ele.nflId==x:
            return ele.outOfBounds

playerProfile['maximumSpeed']=playerProfile.nflId.apply(setMaxSpeed)
playerProfile['maximumAcceleration']=playerProfile.nflId.apply(setMaxAcclr)
playerProfile['tackles']=playerProfile.nflId.apply(setTackle)
playerProfile['quarterbackSacks']=playerProfile.nflId.apply(setqbSack)
playerProfile['passInterceptions']=playerProfile.nflId.apply(setinterceptions)
playerProfile['outOfBounds']=playerProfile.nflId.apply(setOutOfBounds)
## End Code

In [ ]:
## Drop the unrequired birthDate attribute from playerProfile Dataframe

## Start Code
playerProfile=playerProfile.drop(['birthDate'],axis=1)
## End Code

In [ ]:
## Check the Player Profiles
playerProfile

### 2. Plot the Results and Compare the Attributes

In [ ]:
## Check the correlation of each pair of attributes in the playerProfile

## Start Code
correlation=playerProfile.corr()
plt.figure(figsize=(15,10))
sns.heatmap(correlation,annot=True ,cmap = plt.cm.plasma, linecolor='white', linewidths=2)
plt.title('Correlation Between Variables')
plt.show()
plt.savefig('Output8.jpg')
## End Code

#### We can observe a strong positive correlation between the values calculated in the Player Profile.
#### These must be further compared with player characteristics and skills to analyse the relationship

### a. Which players are best against the passing plays ?

In [ ]:
## Extract the top players in Pass Interceptions and sort them by their name

## Start Code
bestPassInterceptors=playerProfile.sort_values('passInterceptions',ascending=False).head(12)
bestPassInterceptors=bestPassInterceptors[['nflId','displayName','position','height','weight','age','passInterceptions']]
bestPassInterceptors=bestPassInterceptors.sort_values('passInterceptions')
## End Code

In [ ]:
## Create a column with a combination of Name and NFL Id to plot it against Pass Interceptions

## Start Code
def func(x):
    return str(x)
bestPassInterceptors['IdName']=bestPassInterceptors.nflId.apply(func)
bestPassInterceptors['IdName']=bestPassInterceptors['displayName']+' ('+bestPassInterceptors['IdName']+')'
## End Code

In [ ]:
## Plot the graph

## Start Code
fig, ax = plt.subplots(figsize=(10,4), dpi= 80)
ax.hlines(y=bestPassInterceptors.IdName,xmin=0,xmax=bestPassInterceptors.passInterceptions,color='firebrick', alpha=0.7, linewidth=2)
ax.scatter(x=bestPassInterceptors.passInterceptions, y=bestPassInterceptors.IdName, s=75, color='firebrick', alpha=0.7)
ax.set_title('Top Players against the Passes', fontdict={'size':16})
ax.set_ylabel('Player Name and NFLId')
ax.set_xlabel('Number of Pass Interceptions')
ax.set_yticks(bestPassInterceptors.IdName)
ax.set_yticklabels(bestPassInterceptors.IdName, fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(0, 30)
for row in bestPassInterceptors[['IdName','passInterceptions','position']].itertuples():
    ax.text(row.passInterceptions+1,row.IdName, s=str(row.passInterceptions)+' '+row.position, horizontalalignment= 'center', verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output9.jpg')
## End Code

#### These are the top 12 players who are the best against the offensive passes. We observe that the defensive position CornerBack

#### is the most occuring position among them. Players at this position contribute more in gaining possession on the ball.

In [ ]:
## Display the Profiles with attributes and their units

## Start Code
df=bestPassInterceptors.copy()
df=df.drop(['IdName'],axis=1)
df.set_index('nflId',inplace=True)
df=df.rename(columns={'height':'height(in inches)','weight':'weight(in pounds)'})
df.sort_values('passInterceptions',ascending=False)
## End Code

In [ ]:
## Extract the top players in Tackles and sort them by their name. Create an extra column
## with a comobination of Name and NFL ID to plot against the Tackles

## Start Code
bestTacklers=playerProfile.sort_values('tackles',ascending=False).head(15)
bestTacklers=bestTacklers[['nflId','displayName','position','height','weight','age','maximumSpeed','maximumAcceleration','tackles']]
bestTacklers=bestTacklers.sort_values('tackles')
def func(x):
    return str(x)
bestTacklers['IdName']=bestTacklers.nflId.apply(func)
bestTacklers['IdName']=bestTacklers['displayName']+' ('+bestTacklers['IdName']+')'
## End Code

In [ ]:
## Plot the graph

## Start Code
fig, ax = plt.subplots(figsize=(10,5), dpi= 80)
ax.hlines(y=bestTacklers.IdName,xmin=0,xmax=bestTacklers.tackles,color='blue', alpha=0.7, linewidth=2)
ax.scatter(x=bestTacklers.tackles, y=bestTacklers.IdName, s=75, color='blue', alpha=0.7)
ax.set_title('Top Players in Tackling', fontdict={'size':16})
ax.set_ylabel('Player Name and NFLId')
ax.set_xlabel('Number of Tackles')
ax.set_yticks(bestTacklers.IdName)
ax.set_yticklabels(bestTacklers.IdName, fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(250, 400)
for row in bestTacklers[['IdName','tackles','position','maximumSpeed','maximumAcceleration']].itertuples():
    ax.text(row.tackles+35,row.IdName, s=str(row.tackles)+' '+row.position+' ('+str(row.maximumSpeed)+' yards/sec,'+str(row.maximumAcceleration)+' yards/sec sq.)', horizontalalignment= 'center', verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output10.jpg')
## End Code

#### These are the top 12 players who are the best in tackling. We observe that the defensive position CornerBack

#### is again the most occuring position among them. Players at this position contribute more in gaining choking the opponent.

In [ ]:
## Display the Profiles with attributes and their units

## Start Code
df=bestTacklers.copy()
df=df.drop(['IdName'],axis=1)
df.set_index('nflId',inplace=True)
df=df.rename(columns={'height':'height(in inches)','weight':'weight(in pounds)','maximumSpeed':'MaxSpeed(in yards/sec)','maximumAcceleration':'MaxAcceleration(in yards/sec sq.)'})
df.sort_values('tackles',ascending=False)
## End Code

### b. Is there anything about a player which can be used to predict his defensive performance ?

In [ ]:
## Select players with more than 150 tackles and sort them with respect to tackles.
## Plot the distribution against the player defensive position

## Start Code
positionTackles=playerProfile[playerProfile['tackles']>150].sort_values('tackles',ascending=False)
positionTackles=pd.DataFrame(positionTackles.groupby('position')['tackles'].count())
fig, ax = plt.subplots(figsize=(12, 7), subplot_kw=dict(aspect="equal"), dpi= 80)
data = positionTackles['tackles']
categories = positionTackles.index
def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.1f}% ({:d} )".format(pct, absolute)
wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),textprops=dict(color="w"), colors=plt.cm.Dark2.colors)
ax.legend(wedges, categories, title="Defensive Positions", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=10, weight=700)
ax.set_title("Positional distribution of Tackles")
plt.show()
plt.savefig('Output11.jpg')
## End Code

#### Position of a player does play an important role in the number of tackles made by him. The above plot shows the distribution of

#### number of players with extra ordinary tackling skills among the defensive positions

In [ ]:
## Select players with more than 150 tackles and sort them with respect to tackles.
## Plot the distribution against the player age

## Start Code
ageTackles=playerProfile[playerProfile['tackles']>150].sort_values('tackles',ascending=False)
ageTackles=pd.DataFrame(ageTackles.groupby('age')['tackles'].count())
fig, ax = plt.subplots(figsize=(12, 7), subplot_kw=dict(aspect="equal"), dpi= 80)
data = ageTackles['tackles']
categories = ageTackles.index
def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.1f}%".format(pct, absolute)
wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),textprops=dict(color="w"), colors=plt.cm.Dark2.colors)
ax.legend(wedges, categories, title="Age", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=10, weight=700)
ax.set_title("Age distribution of Tackles")
plt.show()
plt.savefig('Output12.jpg')
## End Code

#### Here we observe a pattern that the new comers, which are generally in the age range 20 to 22 have a low record in tackling. As

#### age increases, in other words, they gain experience, they improve in tackling. Then again at the ending ages of their career

#### i.e. around 35 to 37 years, we again see a downfall in the number. This is best depicted in the below plot.

In [ ]:
## Plot the frequency of number of players against the ages in a bar fashion

## Start Code
sns.barplot(ageTackles.index,ageTackles['tackles'])
plt.savefig('Output13.jpg')
## End Code

#### Players are best at tackling in the intermediate years if their career. This is usual too. So, the coach can consider a good

#### composition of players of all the age ranges, in appropriate proportion for a balanced defensive team.

In [ ]:
## Box plot the Pass Interceptions distribution among the player positions

## Start Code
sns.set(
    rc={'figure.figsize':(10,5)}, 
    style="white"
)
sns.boxplot(
    x='position',
    y='passInterceptions',
    data=playerProfile
)
sns.despine()
plt.savefig('Output14.jpg')
## End Code

#### In place of considering the mean number of pass interceptions done by players in each defensive position, which might be

#### affected by the outliers, we consider the total distribution and compare between the positions. We see that First Safety provides

#### a good distribution of pass interceptions across all the defensive player positions.

In [ ]:
## Plot a bar graph of distribution of Pass Interceptions among the player heights rounded off to nearest integer

## Start Code
df=playerProfile.copy()
df['height']=round(df['height'])
df=df.rename(columns={'height':'height (in inches)'})
fig = px.bar(df, x="height (in inches)", y="passInterceptions", color="position", barmode = 'stack')
fig.show()
plt.savefig('Output15.jpg')
## End Code

#### Tall players can contribute well in intercepting the passes among the offensive team players. But still, the players in the height

#### range 71 to 73 have a good record in pass interceptions

In [ ]:
## Tabulate the results of Player Profile in a single Data Frame

## Start Code
grpByPos=pd.DataFrame(playerProfile.groupby('position')['tackles','passInterceptions','quarterbackSacks','outOfBounds'].sum())
grpByPos1=pd.DataFrame(playerProfile.groupby('position')['maximumSpeed','maximumAcceleration'].max())
grpByPos=pd.merge(grpByPos,grpByPos1,on="position")
grpByPos=grpByPos.reset_index()
## End Code

grpByPos

## III. Close Trackers of Offensive Pass Receivers

### A player is said to be 'in form' if he provides a consistent performance in consecutive matches. 

### Upto this stage of analysis, we focused on a defensive player's performance across the whole season. 

### Now we concentrate on his performance in a consecutive series of matches to help predict the chances

### of him performing good in the upcoming matches.

### Here, we make use of the targettedReceiversData to get the offensive player to whom the pass was made. 

### 1. Analyse the Games and gather the results.

In [ ]:
## The below code snippet returns a table of combination of defensive players of both the teams in the game with game ID "g"
## sorted by the number of times they have approached the offensive pass receivers very close , number of pass interceptions
## and number of tackles done by them.

## Start Code
def analyseGame(g):
    weekData=weeksData[int(gamesData.loc[gamesData.gameId==g,'week'])-1].copy()
    weekData=weekData[weekData.gameId==g]
    gameTargets=targettedReceiversData[targettedReceiversData.gameId==g].copy()
    gameTargets=gameTargets.reset_index()
    gameTargets=gameTargets.drop(columns=['gameId','index'])
    meanPlayersPos=pd.DataFrame(weekData.groupby(['playId','nflId','position'])[['x','y']].mean())
    meanPlayersPos=meanPlayersPos.reset_index()
    defensivePlayerPositions=list(playerProfile.position.unique())
    plays=list(gameTargets.playId.unique())
    meanPlayersPos.drop(meanPlayersPos[~meanPlayersPos.playId.isin(plays)].index,inplace=True)
    def Keep(id,n,p):
        x=int(gameTargets.loc[gameTargets.playId==id,'nflId'])
        return n==x or (p in defensivePlayerPositions)
    meanPlayersPos['keep']=meanPlayersPos.apply(lambda row:Keep(row.playId,row.nflId,row.position),axis=1)
    meanPlayersPos.drop(meanPlayersPos[meanPlayersPos.keep==False].index,inplace=True)
    meanPlayersPos=meanPlayersPos.reset_index()
    meanPlayersPos=meanPlayersPos.drop(columns=['keep','index'])
    def dist(p,x1,y1):
        n=int(gameTargets.loc[gameTargets.playId==p,'nflId'])
        x=float(meanPlayersPos.loc[(meanPlayersPos.playId==p) & (meanPlayersPos.nflId==n),'x'])
        y=float(meanPlayersPos.loc[(meanPlayersPos.playId==p) & (meanPlayersPos.nflId==n),'y'])
        return (((x-x1)**2)+((y-y1)**2))**0.5
    meanPlayersPos['distance']=meanPlayersPos.apply(lambda row:dist(row.playId,row.x,row.y),axis=1)
    meanPlayersPos.drop(meanPlayersPos[meanPlayersPos.distance==0].index,inplace=True)
    meanPlayersPos=meanPlayersPos.sort_values(['playId','distance'])
    meanPlayersPos=meanPlayersPos.reset_index()
    meanPlayersPos=meanPlayersPos.drop(columns=['index'])
    def countPassIntercepts(n):
        df=weekData[(weekData.nflId==n) & (weekData.event=='pass_outcome_interception')]
        return len(df.axes[0])
    def countTackles(n):
        df=weekData[(weekData.nflId==n) & (weekData.event=='tackle')]
        return len(df.axes[0])
    playerPerformance=pd.DataFrame(meanPlayersPos.groupby('nflId')['playId'].count())
    playerPerformance=playerPerformance.reset_index()
    playerPerformance.rename(columns={'playId':'closeTrackings'},inplace=True)
    playerPerformance['passInterceptions']=playerPerformance.nflId.apply(countPassIntercepts)
    playerPerformance['tackles']=playerPerformance.nflId.apply(countTackles)
    playerPerformance=playerPerformance.sort_values(['closeTrackings','passInterceptions','tackles'],ascending=False)
    playerPerformance=playerPerformance.merge(playersData[['nflId','displayName','age','position','height','weight']],on='nflId',how='left')
    return playerPerformance
## End Code

### We randomly chose 5 teams and analyse the first 5 games played by them. The 5 teams emerged to be :-

#### 1. Green Bay Packers (GB)
#### 2. Pittsburg Steelers (PIT)
#### 3. Tennessee Titans (TEN)
#### 4. Oakland Raiders (OAK)
#### 5. Minnesota Vikings (MIN)

In [ ]:
## Randomly select 5 distinct games played by the team GB, and analyse each game. Store the results in a list.
## Count the number of times each player of team GB in the result have appeared in the top 10 of the result.
## Sort the players according to this count.

## Start Code
gamesGB=list(gamesData.loc[(gamesData.homeTeamAbbr=='GB') | (gamesData.visitorTeamAbbr=='GB')]['gameId'])
gamesGB=gamesGB[:5]
gameAnalysesGB=[]
for gameGB in gamesGB:
    gameAnalysesGB.append(analyseGame(gameGB))
def topped(id):
    count=0
    for gameAnalysisGB in gameAnalysesGB:
        gameAnalysisGB=gameAnalysisGB[:10]
        if id in list(gameAnalysisGB.nflId):
            count+=1
    return count
topPerformersGB=pd.merge(gameAnalysesGB[0][['nflId','displayName','height','weight','age','position']],gameAnalysesGB[1][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersGB=pd.merge(topPerformersGB[['nflId','displayName','height','weight','age','position']],gameAnalysesGB[2][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersGB=pd.merge(topPerformersGB[['nflId','displayName','height','weight','age','position']],gameAnalysesGB[3][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersGB=pd.merge(topPerformersGB[['nflId','displayName','height','weight','age','position']],gameAnalysesGB[4][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersGB['topEncounters']=topPerformersGB.nflId.apply(topped)
topPerformersGB=topPerformersGB.sort_values('topEncounters',ascending=False)
topPerformersGB
## End Code

In [ ]:
## Randomly select 5 distinct games played by the team PIT, and analyse each game. Store the results in a list.
## Count the number of times each player of team PIT in the result have appeared in the top 10 of the result.
## Sort the players according to this count.

## Start Code
gamesPIT=list(gamesData.loc[(gamesData.homeTeamAbbr=='PIT') | (gamesData.visitorTeamAbbr=='PIT')]['gameId'])
gamesPIT=gamesPIT[:5]
gameAnalysesPIT=[]
for gamePIT in gamesPIT:
    gameAnalysesPIT.append(analyseGame(gamePIT))
def topped(id):
    count=0
    for gameAnalysisPIT in gameAnalysesPIT:
        gameAnalysisPIT=gameAnalysisPIT[:10]
        if id in list(gameAnalysisPIT.nflId):
            count+=1
    return count
topPerformersPIT=pd.merge(gameAnalysesPIT[0][['nflId','displayName','height','weight','age','position']],gameAnalysesPIT[1][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersPIT=pd.merge(topPerformersPIT[['nflId','displayName','height','weight','age','position']],gameAnalysesPIT[2][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersPIT=pd.merge(topPerformersPIT[['nflId','displayName','height','weight','age','position']],gameAnalysesPIT[3][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersPIT=pd.merge(topPerformersPIT[['nflId','displayName','height','weight','age','position']],gameAnalysesPIT[4][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersPIT['topEncounters']=topPerformersPIT.nflId.apply(topped)
topPerformersPIT=topPerformersPIT.sort_values('topEncounters',ascending=False)
topPerformersPIT
## End Code

In [ ]:
## Randomly select 5 distinct games played by the team TEN, and analyse each game. Store the results in a list.
## Count the number of times each player of team TEN in the result have appeared in the top 10 of the result.
## Sort the players according to this count.

## Start Code
gamesTEN=list(gamesData.loc[(gamesData.homeTeamAbbr=='TEN') | (gamesData.visitorTeamAbbr=='TEN')]['gameId'])
gamesTEN=gamesTEN[:5]
gameAnalysesTEN=[]
for gameTEN in gamesTEN:
    gameAnalysesTEN.append(analyseGame(gameTEN))
def topped(id):
    count=0
    for gameAnalysisTEN in gameAnalysesTEN:
        gameAnalysisTEN=gameAnalysisTEN[:10]
        if id in list(gameAnalysisTEN.nflId):
            count+=1
    return count
topPerformersTEN=pd.merge(gameAnalysesTEN[0][['nflId','displayName','height','weight','age','position']],gameAnalysesTEN[1][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersTEN=pd.merge(topPerformersTEN[['nflId','displayName','height','weight','age','position']],gameAnalysesTEN[2][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersTEN=pd.merge(topPerformersTEN[['nflId','displayName','height','weight','age','position']],gameAnalysesTEN[3][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersTEN=pd.merge(topPerformersTEN[['nflId','displayName','height','weight','age','position']],gameAnalysesTEN[4][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersTEN['topEncounters']=topPerformersTEN.nflId.apply(topped)
topPerformersTEN=topPerformersTEN.sort_values('topEncounters',ascending=False)
topPerformersTEN
## End Code

In [ ]:
## Randomly select 5 distinct games played by the team OAK, and analyse each game. Store the results in a list.
## Count the number of times each player of team OAK in the result have appeared in the top 10 of the result.
## Sort the players according to this count.

## Start Code
gamesOAK=list(gamesData.loc[(gamesData.homeTeamAbbr=='OAK') | (gamesData.visitorTeamAbbr=='OAK')]['gameId'])
gamesOAK=gamesOAK[:5]
gameAnalysesOAK=[]
for gameOAK in gamesOAK:
    gameAnalysesOAK.append(analyseGame(gameOAK))
def topped(id):
    count=0
    for gameAnalysisOAK in gameAnalysesOAK:
        gameAnalysisOAK=gameAnalysisOAK[:10]
        if id in list(gameAnalysisOAK.nflId):
            count+=1
    return count
topPerformersOAK=pd.merge(gameAnalysesOAK[0][['nflId','displayName','height','weight','age','position']],gameAnalysesOAK[1][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersOAK=pd.merge(topPerformersOAK[['nflId','displayName','height','weight','age','position']],gameAnalysesOAK[2][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersOAK=pd.merge(topPerformersOAK[['nflId','displayName','height','weight','age','position']],gameAnalysesOAK[3][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersOAK=pd.merge(topPerformersOAK[['nflId','displayName','height','weight','age','position']],gameAnalysesOAK[4][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersOAK['topEncounters']=topPerformersOAK.nflId.apply(topped)
topPerformersOAK=topPerformersOAK.sort_values('topEncounters',ascending=False)
topPerformersOAK
## End Code

In [ ]:
## Randomly select 5 distinct games played by the team MIN, and analyse each game. Store the results in a list.
## Count the number of times each player of team MIN in the result have appeared in the top 10 of the result.
## Sort the players according to this count.

## Start Code
gamesMIN=list(gamesData.loc[(gamesData.homeTeamAbbr=='MIN') | (gamesData.visitorTeamAbbr=='MIN')]['gameId'])
gamesMIN=gamesMIN[:5]
gameAnalysesMIN=[]
for gameMIN in gamesMIN:
    gameAnalysesMIN.append(analyseGame(gameMIN))
def topped(id):
    count=0
    for gameAnalysisMIN in gameAnalysesMIN:
        gameAnalysisMIN=gameAnalysisMIN[:10]
        if id in list(gameAnalysisMIN.nflId):
            count+=1
    return count
topPerformersMIN=pd.merge(gameAnalysesMIN[0][['nflId','displayName','height','weight','age','position']],gameAnalysesMIN[1][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersMIN=pd.merge(topPerformersMIN[['nflId','displayName','height','weight','age','position']],gameAnalysesMIN[2][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersMIN=pd.merge(topPerformersMIN[['nflId','displayName','height','weight','age','position']],gameAnalysesMIN[3][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersMIN=pd.merge(topPerformersMIN[['nflId','displayName','height','weight','age','position']],gameAnalysesMIN[4][['nflId','displayName','height','weight','age','position']],how='inner',on=['nflId','displayName','height','weight','age','position'])
topPerformersMIN['topEncounters']=topPerformersMIN.nflId.apply(topped)
topPerformersMIN=topPerformersMIN.sort_values('topEncounters',ascending=False)
topPerformersMIN
## End Code

### 2. Plot the Results and Compare the Attributes

In [ ]:
## Plot the graph

## Start Code
def convert(x):
    return str(x)
topPerformersGB1=topPerformersGB.copy()
topPerformersGB1['IdName']=topPerformersGB1.nflId.apply(convert)
topPerformersGB1['IdName']=topPerformersGB1['displayName']+' ('+topPerformersGB1['IdName']+')'
topPerformersGB1=topPerformersGB1.sort_values('topEncounters')
fig, ax = plt.subplots(figsize=(10,5), dpi= 80)
ax.hlines(y=topPerformersGB1.IdName,xmin=0,xmax=topPerformersGB1.topEncounters,color='firebrick',alpha=0.7,linewidth=2)
ax.scatter(x=topPerformersGB1.topEncounters,y=topPerformersGB1.IdName,s=75,color='firebrick', alpha=0.7)
ax.set_title('Top Performing Players of GB', fontdict={'size':16})
ax.set_ylabel('Player Name and nflId')
ax.set_xlabel('Number of Top Encounters')
ax.set_yticks(topPerformersGB1.IdName)
ax.set_yticklabels(topPerformersGB1.IdName,fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(0,10)
for row in topPerformersGB1[['IdName','topEncounters','position']].itertuples():
    ax.text(row.topEncounters+0.5,row.IdName, s=str(row.topEncounters)+' '+row.position,horizontalalignment='center',verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output16.jpg')
## End Code

#### These are the rank wise players who have been consistently performing well in matches of the team Green Bay Packers. They 
#### are expected to play well in the upcoming matches too.

In [ ]:
## Plot the graph

## Start Code
def convert(x):
    return str(x)
topPerformersPIT1=topPerformersPIT.copy()
topPerformersPIT1['IdName']=topPerformersPIT1.nflId.apply(convert)
topPerformersPIT1['IdName']=topPerformersPIT1['displayName']+' ('+topPerformersPIT1['IdName']+')'
topPerformersPIT1=topPerformersPIT1.sort_values('topEncounters')
fig, ax = plt.subplots(figsize=(10,5), dpi= 80)
ax.hlines(y=topPerformersPIT1.IdName,xmin=0,xmax=topPerformersPIT1.topEncounters,color='firebrick',alpha=0.7,linewidth=2)
ax.scatter(x=topPerformersPIT1.topEncounters,y=topPerformersPIT1.IdName,s=75,color='firebrick', alpha=0.7)
ax.set_title('Top Performing Players of PIT', fontdict={'size':16})
ax.set_ylabel('Player Name and nflId')
ax.set_xlabel('Number of Top Encounters')
ax.set_yticks(topPerformersPIT1.IdName)
ax.set_yticklabels(topPerformersPIT1.IdName,fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(0,10)
for row in topPerformersPIT1[['IdName','topEncounters','position']].itertuples():
    ax.text(row.topEncounters+0.5,row.IdName, s=str(row.topEncounters)+' '+row.position,horizontalalignment='center',verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output17.jpg')
## End Code

#### These are the rank wise players who have been consistently performing well in matches of the team Piitsburg Steelers. They 
#### are expected to play well in the upcoming matches too.

In [ ]:
## Plot the graph

## Start Code
def convert(x):
    return str(x)
topPerformersTEN1=topPerformersTEN.copy()
topPerformersTEN1['IdName']=topPerformersTEN1.nflId.apply(convert)
topPerformersTEN1['IdName']=topPerformersTEN1['displayName']+' ('+topPerformersTEN1['IdName']+')'
topPerformersTEN1=topPerformersTEN1.sort_values('topEncounters')
fig, ax = plt.subplots(figsize=(10,5), dpi= 80)
ax.hlines(y=topPerformersTEN1.IdName,xmin=0,xmax=topPerformersTEN1.topEncounters,color='firebrick',alpha=0.7,linewidth=2)
ax.scatter(x=topPerformersTEN1.topEncounters,y=topPerformersTEN1.IdName,s=75,color='firebrick', alpha=0.7)
ax.set_title('Top Performing Players of TEN', fontdict={'size':16})
ax.set_ylabel('Player Name and nflId')
ax.set_xlabel('Number of Top Encounters')
ax.set_yticks(topPerformersTEN1.IdName)
ax.set_yticklabels(topPerformersTEN1.IdName,fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(0,10)
for row in topPerformersTEN1[['IdName','topEncounters','position']].itertuples():
    ax.text(row.topEncounters+0.5,row.IdName, s=str(row.topEncounters)+' '+row.position,horizontalalignment='center',verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output18.jpg')
## End Code

#### These are the rank wise players who have been consistently performing well in matches of the team Tennessee Titans. They 
#### are expected to play well in the upcoming matches too.

In [ ]:
## Plot the graph

## Start Code
def convert(x):
    return str(x)
topPerformersOAK1=topPerformersOAK.copy()
topPerformersOAK1['IdName']=topPerformersOAK1.nflId.apply(convert)
topPerformersOAK1['IdName']=topPerformersOAK1['displayName']+' ('+topPerformersOAK1['IdName']+')'
topPerformersOAK1=topPerformersOAK1.sort_values('topEncounters')
fig, ax = plt.subplots(figsize=(10,5), dpi= 80)
ax.hlines(y=topPerformersOAK1.IdName,xmin=0,xmax=topPerformersOAK1.topEncounters,color='firebrick',alpha=0.7,linewidth=2)
ax.scatter(x=topPerformersOAK1.topEncounters,y=topPerformersOAK1.IdName,s=75,color='firebrick', alpha=0.7)
ax.set_title('Top Performing Players of OAK', fontdict={'size':16})
ax.set_ylabel('Player Name and nflId')
ax.set_xlabel('Number of Top Encounters')
ax.set_yticks(topPerformersOAK1.IdName)
ax.set_yticklabels(topPerformersOAK1.IdName,fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(0,10)
for row in topPerformersOAK1[['IdName','topEncounters','position']].itertuples():
    ax.text(row.topEncounters+0.5,row.IdName, s=str(row.topEncounters)+' '+row.position,horizontalalignment='center',verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output19.jpg')
## End Code

#### These are the rank wise players who have been consistently performing well in matches of the team Oakland Raiders. They 
#### are expected to play well in the upcoming matches too.

In [ ]:
## Plot the graph

## Start Code
def convert(x):
    return str(x)
topPerformersMIN1=topPerformersMIN.copy()
topPerformersMIN1['IdName']=topPerformersMIN1.nflId.apply(convert)
topPerformersMIN1['IdName']=topPerformersMIN1['displayName']+' ('+topPerformersMIN1['IdName']+')'
topPerformersMIN1=topPerformersMIN1.sort_values('topEncounters')
fig, ax = plt.subplots(figsize=(10,5), dpi= 80)
ax.hlines(y=topPerformersMIN1.IdName,xmin=0,xmax=topPerformersMIN1.topEncounters,color='firebrick',alpha=0.7,linewidth=2)
ax.scatter(x=topPerformersMIN1.topEncounters,y=topPerformersMIN1.IdName,s=75,color='firebrick', alpha=0.7)
ax.set_title('Top Performing Players of MIN', fontdict={'size':16})
ax.set_ylabel('Player Name and nflId')
ax.set_xlabel('Number of Top Encounters')
ax.set_yticks(topPerformersMIN1.IdName)
ax.set_yticklabels(topPerformersMIN1.IdName,fontdict={'horizontalalignment': 'right', 'size':12})
ax.set_xlim(0,10)
for row in topPerformersMIN1[['IdName','topEncounters','position']].itertuples():
    ax.text(row.topEncounters+0.5,row.IdName, s=str(row.topEncounters)+' '+row.position,horizontalalignment='center',verticalalignment='bottom', fontsize=10)
plt.show()
plt.savefig('Output20.jpg')
## End Code

#### These are the rank wise players who have been consistently performing well in matches of the team Minnesota Vikings. They 
#### are expected to play well in the upcoming matches too.

### The above analysis can be replicated to any team and any game played by them. The snippet of code 
### provided is completely generalised to yield the top players of any game, from both the teams playing the
### match, provided the game id of the match. 

# Conclusion
### This particular player profile approach sums up the need of analysis in the area of defensive analysis by
### providing complete information of the players through their profiles, along with answering important
### questions that help analyse and evaluate defensive performances of those players.
### The Player Profile approach collectively provides every player information that is necessary for the game
### analysis and that in turn helps the team coach or manager to evaluate, analyze or set up a good defensive
### strategy.
### The futher part of our analysis provides the flow of the top players performance in any given team and help 
### to predict how they play in the further coming matches and decide to include them or not for a strong
### defensive strategy.